In [27]:
!pip install soynlp -q

In [28]:
from google.colab import drive

drive.mount('/content/drive') #구글 드라이브에 올리면 실행 가능

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
import re
import json
from soynlp.normalizer import *

In [30]:
def PreProcess(text):
    bad_chars = {"\u200b": "", "…": "...", "\ufeff": ""}
    for bad_char in bad_chars:
        text = text.replace(bad_char, bad_chars[bad_char]) # 문제를 일으킬 수 있는 문자 제거(너비가 0인 공백문자,인코딩 정보를 알려주는 역할 문자)
    error_chars = {"\u3000": " ", "\u2009": " ", "\u2002": " ", "\xa0":" "} #(표의 공백문자, 한칸, 엔터+스페이스, 빈 공백-유니코드에서 )
    for error_char in error_chars:
        text = text.replace(error_char, error_chars[error_char])
    text = re.sub(pattern = '^[a-zA-Z0-9]+@[a-zA-Z0-9]+$', \
                  repl ='', string = text) #이메일 제거
    text = re.sub(pattern = '#\S+', \
                  repl ='', string = text) # "#문자" 형식 어절 제거
    text = re.sub(pattern = '@\w+', \
                  repl ='', string = text) # "@문자" 형식 어절 제거
    text = re.sub(pattern = '\(출처 ?= ?.+\) |\(사진 ?= ?.+\) |\(자료 ?= ?.+\)| \(자료사진\) |사진=.+기자 ', \
                  repl ='', string = text) # 뉴스 내 포함된 이미지에 대한 레이블 제거
    _filter = re.compile('https?://\S+|www\.\S+')
    text = _filter.sub('', text) #url 전처리
    _filter = re.compile('[ㄱ-ㅣ]+') 
    text = _filter.sub('', text) #자모음제거
    _filter = re.compile('.co\S')
    text = _filter.sub('', text) #.co 전처리
    _filter = re.compile('[一-龥]')
    text = _filter.sub('', text) #한자 전처리  
    _filter = re.compile('[^가-힣 0-9 a-z A-Z \. \n \, \' \" \? \! \< \> \: \《 \》\『\』 \-]+') 
    text = _filter.sub('', text) #(".",",","'","?","!")를 제외한 특수문자 제거
    # if "\n" in text:
    #     print("fail")
    return text

In [32]:
#save_data: 데이터를 합쳐서 저장하는 함수

from glob import glob

def save_data(file_path, save_path):
    print('{} data saving.'.format(file_path.split('/')[-1]))
    save_path_1 = '/content/drive/MyDrive/Bookathon/글틴2_서윤.json'
    with open(file_path, 'r') as f:
        json_data = json.load(f)
        text = json_data['data']
        length = len(text)
        for i in range (length):
            text[i]['text'] = PreProcess(text[i]["text"])
    data={}
    data["data"] = text

    with open(save_path_1, 'w') as outfile:
        json.dump(data, outfile, ensure_ascii=False)    
      


    print("\nAll saved.".format(file_path.split('/')[-1]))

    return data

In [36]:
#합칠 데이터와 새로 저장할 데이터 Path 지정 => 전처리할 데이터와 전처리 저장할 데이터에 맞는 위치에 맞게 변경

file_1 = '/content/drive/MyDrive/Bookathon/글틴2_서윤.json' #크롤링 후 수집한 데이터의 위치(전처리 이전) => 팀 데이터 보관위치에 맞게 변경
save_path_1 = '/content/drive/MyDrive/Bookathon/글틴_2_서윤_기계처리 + 사람처리.json' #저장할 데정터 위치(전처리된) => 팀 데이터 보관할 위치에 맞게 변경

In [37]:
#함수 실행

data = save_data(file_1, save_path_1)

창훈_글틴2_전처리 전.json data saving.

All saved.


In [25]:
print(data["data"][0])

NameError: ignored